In [20]:
import numpy as np              # Python's standard numerical library
import matplotlib.pyplot as plt # Python's scientific visualization library
import pyidi                    # Python HSC data analysis library
import pickle as pk
import os
from pixel_setter2 import PixelSetter#, play_video, detect_peaks
from pixel_setter import play_video
from skimage import feature, measure
import importlib
from EMA_functions import *
import matplotlib.animation as animation
root_drive_sim = os.path.normpath('G:/.shortcut-targets-by-id/1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx/Illimited Lab Projects/Spiders/Simulations')

### Import video

In [3]:
# file_name = "Full_web_ecc0_new_Floc4_v1" # First attempt
file_name = "Full_web_ecc0_new_Floc9_v1"    # Second attempt, ecc0
# file_name = "Full_web_ecc1_new_Floc1_v5"      # Second attempt, ecc1
# file_name = "Full_web_ecc2_new_Floc7_v1"        # Second attempt, ecc2
folder_paths = ['H:/My Drive/PHD/HSC', 'D:/HSC', 'F:/', 'E:/thijs/', 'C:/Users/thijs/Documents/HSC/', 'D:/thijsmas/HSC']
video = open_video(file_name, folder_paths)

H:/My Drive/PHD/HSC\ecc0\web0\Full_web_ecc0_new_Floc9_v1_S01\Full_web_ecc0_new_Floc9_v1_S01.cihx


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\pyMRAW.py:103: UserWarning: Clipped footage! (Total frame: 10000, Original total frame: 87371)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))


### Plot still frame in video

In [4]:
%matplotlib qt
sequential_image_n = 0
still_image = video.mraw[sequential_image_n]
plot_still_frame(video, sequential_image_n, show_saturation=True)
first_frame = 200
ani = play_video(video, range(first_frame,video.N), interval=30)

### Set pixels

In [53]:
%matplotlib qt
pix_set = PixelSetter(still_image, file_name = file_name)
pix_set.set_neighborhood_size(4)
pix_set.choose_reference_centers()


### Combine the tracking points and exclude erroneous points

In [11]:
# pix_set.combine_tracking_points()
# tp = np.array(list(pix_set.tracking_points))
center = np.array(still_image.shape[:2])/2
center[1] = center[1] + 45
center[0] = center[0] - 30
radius = 0.88*np.linalg.norm(center)
distances = np.linalg.norm(tp - center, axis=1)
inside_web = (distances < radius)
hor_section = (tp[:, 1] > 15) & (tp[:, 1] < 1100)
ver_section = (tp[:, 0] > 15) & (tp[:, 0] < 450)
tp_lim = tp[hor_section & ver_section & inside_web]

fig, ax = plt.subplots(figsize=(10, 10))

# plot tp
plt.scatter(center[1], center[0])
plt.scatter(tp[:, 1], tp[:, 0], c='red', s=10, label='tp')

# plot tp_lim with markers on top of tp
plt.scatter(tp_lim[:, 1], tp_lim[:, 0], c='green', s=5, label='tp_lim')
ax.set_xlim([0, still_image.shape[1]])
ax.set_ylim([still_image.shape[0] , 0])
ax.axis('equal')
# show the plot
plt.show()


### Save or open tracking points

In [9]:
# np.savetxt(f'tp_{file_name}.txt', tp, fmt='%.18e', delimiter=',')
tp = np.loadtxt(f'tp_{file_name}.txt', delimiter=',')

In [56]:
video.set_points(tp_lim)
video.set_method('lk')
mraw_range = (sequential_image_n+1, 500)
reference_image = (0, 30)
roi_size_vec = [(3, 3), (5,5), (7, 7), (9, 9), (11, 11), (13, 13)]
for roi_size in roi_size_vec:
    video.method.configure(mraw_range=mraw_range, reference_image = reference_image, roi_size= roi_size)
    video.show_points()

    d = video.get_displacements(processes = -1)
    d = video.displacements

    root = 'C:/Users/thijsmas/Documents/GitHub/pyidi_data/'
    path = os.path.join(root, file_name+f'_d_rs{str(roi_size)}_ri{str(reference_image)}'+'.pkl')
    with open(path, 'wb') as f:
        pk.dump({'displacement':d, 'tracking points': tp_lim}, f)
    
    dd = np.diff(d, axis=1)
    last_10_zeros = np.any(dd[:, -10:, :] == 0, axis=(1,2))
    large_d = np.any(np.linalg.norm(d, axis=2) > 10, axis = 1)
    good_points = np.logical_not(last_10_zeros | large_d)

    first_frame = 1

    td = d +  tp_lim.reshape(len(tp_lim),1,2)
    ani = play_video(video, range(first_frame, 497), interval=30, points=td[good_points, first_frame:, :])
    writer = animation.FFMpegWriter(fps=60)
    ani.save(os.path.join(root_drive_sim,'anim', f'{file_name}_d_rs{str(roi_size)}_ri{str(reference_image)}_anim_.mp4'), writer=writer)



### Extract displacements

In [57]:
d = video.get_displacements(processes = -1)
# video.method.valid_points
d = video.displacements

--- Starting new analysis ---
 
Computation start: 2023-11-21 14:37:17.001965


<frozen importlib._bootstrap>:1047: ImportWarning: ImportDenier.find_spec() not found; falling back to find_module()
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


VBox()

Computation duration: 00:03:9.30
H:/My Drive/PHD/HSC\ecc0\web0\Full_web_ecc0_new_Floc9_v1_S01\Full_web_ecc0_new_Floc9_v1_S01_pyidi_analysis\analysis_008\ analysis_008


### Save or open displacements and tracking points

In [58]:
root = 'C:/Users/thijsmas/Documents/GitHub/pyidi_data/'
path = os.path.join(root, file_name+f'_d_rs{str(roi_size)}_ri{str(reference_image)}'+'.pkl')
with open(path, 'wb') as f:
    pk.dump({'displacement':d, 'tracking points': tp_lim}, f)

### Animate the results

In [52]:
%matplotlib qt
dd = np.diff(d, axis=1)
last_10_zeros = np.any(dd[:, -10:, :] == 0, axis=(1,2))
large_d = np.any(np.linalg.norm(d, axis=2) > 10, axis = 1)
good_points = np.logical_not(last_10_zeros | large_d)

first_frame = 1

td = d +  tp_lim.reshape(len(tp_lim),1,2)
ani = play_video(video, range(first_frame, 497), interval=30, points=td[good_points, first_frame:, :])
writer = animation.FFMpegWriter(fps=60)
ani.save(os.path.join(root_drive_sim,'anim', f'{file_name}_d_rs{str(roi_size)}_ri{str(reference_image)}_anim_.mp4'), writer=writer)

<frozen importlib._bootstrap>:1047: ImportWarning: ImportDenier.find_spec() not found; falling back to find_module()


IndexError: index 498 is out of bounds for axis 1 with size 498

### Plot signals

In [28]:
fig, ax = plt.subplots()
for point in np.arange(d.shape[0])[good_points][::5]: #[video.method.valid_points]
    ax.plot(np.arange(video.N), d[point, :, 0], label='point {} - dir 0'.format(point)) #video.info['Record Rate(fps)']*
    ax.plot(np.arange(video.N), d[point, :, 1], label='point {} - dir 1'.format(point)) #video.info['Record Rate(fps)']*
# ax.legend()
# ax.set_ylim(-50, 50)
ax.set_xlabel('frame number')
ax.set_ylabel('displacement [pixels]')

Text(0, 0.5, 'displacement [pixels]')